# NHL Game Prediction Modeling
by Gary Schwaeber

## Overview

With sport betting becoming increasingly popular and mainstream I believe that data science can be used to make superior decisions over gut intuitions. In this notebook I will attempt to train logistic regression, ada boost, and gradient boosting models in an attempt to make the best possible game prediction model. I will train my models and tune model hyperparemetres using game results from seasons '2017-2018', '2018-2019', '2019-2020'. Then I will predict on held out games from the current 2021 season and evaluate my model. There are currently a handful of public models whose log loss on the current season's games is being [tracked](https://hockey-statistics.com/2021/05/03/game-projections-january-13th-2021/) on which I can compare the quality of my model to. The score I will look to optimize is log loss, however, I will also review accuracy scores due to their interpretability.

Log-loss is indicative of how close the prediction probability is to the corresponding actual/true value (0 or 1 in case of binary classification). The more the predicted probability diverges from the actual value, the higher is the log-loss value. [Source](https://towardsdatascience.com/intuition-behind-log-loss-score-4e0c9979680a)


In [275]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.preprocessing import normalize, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, auc

from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import RFECV

#for the Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping
from keras.constraints import maxnorm

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('data/all_games_multirolling_SVA_2.csv')

In [4]:
df.shape

(4447, 155)

In [29]:
# define feature columns for different rolling intervals

common = ['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 
 'home_Rating.A.Pre',
 'away_Rating.A.Pre',
 'B2B_Status']

r3 = ['home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_xGF_per_min_pp',
 'home_last3_GF_per_min_pp',
 'home_last3_xGA_per_min_pk',
 'home_last3_GA_per_min_pk',
 'away_last_3_FF%_5v5',
 'away_last_3_GF%_5v5',
 'away_last_3_xGF%_5v5',
 'away_last_3_SH%',
 'away_last3_xGF_per_min_pp',
 'away_last3_GF_per_min_pp',
 'away_last3_xGA_per_min_pk',
 'away_last3_GA_per_min_pk'] + common

r5 =['home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',

 'home_last5_xGF_per_min_pp',
 'home_last5_GF_per_min_pp',

 'home_last5_xGA_per_min_pk',
 'home_last5_GA_per_min_pk',
 'away_last_5_FF%_5v5',
 'away_last_5_GF%_5v5',
 'away_last_5_xGF%_5v5',
 'away_last_5_SH%',
 'away_last5_xGF_per_min_pp',
 'away_last5_GF_per_min_pp',
 'away_last5_xGA_per_min_pk',
 'away_last5_GA_per_min_pk'] + common

r10 =['home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_xGF_per_min_pp',
 'home_last10_GF_per_min_pp',
 'home_last10_xGA_per_min_pk',
 'home_last10_GA_per_min_pk',
  'away_last_10_FF%_5v5',
 'away_last_10_GF%_5v5',
 'away_last_10_xGF%_5v5',
 'away_last_10_SH%',
 'away_last10_xGF_per_min_pp',
 'away_last10_GF_per_min_pp',
 'away_last10_xGA_per_min_pk',
 'away_last10_GA_per_min_pk',]


r20 = ['home_last_20_FF%_5v5',
 'home_last_20_GF%_5v5',
 'home_last_20_xGF%_5v5',
 'home_last_20_SH%',

 'home_last20_xGF_per_min_pp',
 'home_last20_GF_per_min_pp',

 'home_last20_xGA_per_min_pk',
 'home_last20_GA_per_min_pk',
 'away_last_20_FF%_5v5',
 'away_last_20_GF%_5v5',
 'away_last_20_xGF%_5v5',
 'away_last_20_SH%',

 'away_last20_xGF_per_min_pp',
 'away_last20_GF_per_min_pp',

 'away_last20_xGA_per_min_pk',
 'away_last20_GA_per_min_pk']

r30 = ['home_last_30_FF%_5v5',
 'home_last_30_GF%_5v5',
 'home_last_30_xGF%_5v5',
 'home_last_30_SH%',
 'home_last30_xGF_per_min_pp',
 'home_last30_GF_per_min_pp',
 'home_last30_xGA_per_min_pk',
 'home_last30_GA_per_min_pk',
 'away_last_30_FF%_5v5',
 'away_last_30_GF%_5v5',
 'away_last_30_xGF%_5v5',
 'away_last_30_SH%',
 'away_last30_xGF_per_min_pp',
 'away_last30_GF_per_min_pp',
 'away_last30_xGA_per_min_pk',
 'away_last30_GA_per_min_pk'] + common


r40 = ['home_last_40_FF%_5v5',
 'home_last_40_GF%_5v5',
 'home_last_40_xGF%_5v5',
 'home_last_40_SH%',
 'home_last40_xGF_per_min_pp',
 'home_last40_GF_per_min_pp',
 'home_last40_xGA_per_min_pk',
 'home_last40_GA_per_min_pk',
 'away_last_40_FF%_5v5',
 'away_last_40_GF%_5v5',
 'away_last_40_xGF%_5v5',
 'away_last_40_SH%',
 'away_last40_xGF_per_min_pp',
 'away_last40_GF_per_min_pp',
 'away_last40_xGA_per_min_pk',
 'away_last40_GA_per_min_pk'] + common


all_r = list(set(r3+r5+r10+r20+r30+r40))

r3_30 =list(set(r3+r30))
r5_30 = list(set(r5+r30))
r10_30 = list(set(r10+r30))
r_3_5_30 = list(set(r3+r5+r30))
r_5_20 = list(set(r5+r20))
r_5_40 = list(set(r5+r40))

## Baseline Model

The baseline model will predict that every home team wins their game and that the probability of that is the ratio of games the home team has won.

In [6]:
df['Home_Team_Won'].value_counts(normalize=True)

1    0.541714
0    0.458286
Name: Home_Team_Won, dtype: float64

In [7]:
baseline_preds = np.ones(df.shape[0])
accuracy_score(df['Home_Team_Won'],baseline_preds)

0.5417135147290308

In [8]:
baseline_probs = np.repeat(df['Home_Team_Won'].value_counts(normalize=True)[1], df.shape[0])

log_loss(df['Home_Team_Won'], baseline_probs)

0.6896630977766495

The models will need to beat an accuracy score of 54.17% and a log loss of .6897, otherwise they are no better than just predicting the home team will win. 

## Rolling 5 and 40 game features

For my first set of models I will attempt using 5 and 40 game rolling features. These seemed like a good set based on the feature selection notebook. 40 games is currently the longest rolling runway I have currently for the team statistics. The 40 games stats intuitively provide the most smoothing of team data over the course of the season, while the 5 game stats may provide some insight on any streakiness or may cover recent developments that would affect short term team performances such as player injuries, trades coaching changes etc.

In [87]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r_5_40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,r_5_40]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [26]:
X_train.columns

Index(['home_last40_xGF_per_min_pp', 'away_last_5_xGF%_5v5',
       'home_last5_pp_TOI_per_game', 'home_last_40_GF%_5v5',
       'home_last40_xGA_per_min_pk', 'home_last5_xGA_per_min_pk',
       'home_last_40_SH%', 'home_last5_pk_TOI_per_game',
       'away_last40_pp_TOI_per_game', 'home_Goalie_GSAx/60',
       'away_last40_pk_TOI_per_game', 'away_Goalie_GSAx/60',
       'away_last_5_GF%_5v5', 'home_last40_pk_TOI_per_game', 'B2B_Status',
       'home_last_40_xGF%_5v5', 'away_last5_pp_TOI_per_game',
       'away_last5_pk_TOI_per_game', 'home_last5_GF_per_min_pp',
       'home_last_5_GF%_5v5', 'home_last_5_FF%_5v5',
       'away_last5_xGF_per_min_pp', 'away_last40_xGF_per_min_pp',
       'home_last40_GA_per_min_pk', 'home_Goalie_HDCSV%',
       'away_last5_GA_per_min_pk', 'away_last40_GF_per_min_pp',
       'away_Rating.A.Pre', 'home_last_5_xGF%_5v5', 'away_last_5_SH%',
       'home_Rating.A.Pre', 'home_last5_xGF_per_min_pp',
       'away_last_40_xGF%_5v5', 'home_last5_GA_per_min_pk',
  

In [10]:
X_train.shape

(3582, 49)

In [23]:
numeric_features = ['home_last40_xGF_per_min_pp', 'away_last_5_xGF%_5v5',
       'home_last_40_GF%_5v5',
       'home_last40_xGA_per_min_pk', 'home_last5_xGA_per_min_pk',
       'home_last_40_SH%', 
       'home_Goalie_GSAx/60',
        'away_Goalie_GSAx/60',
       'away_last_5_GF%_5v5', 
       'home_last_40_xGF%_5v5', 
     'home_last5_GF_per_min_pp',
       'home_last_5_GF%_5v5', 'home_last_5_FF%_5v5',
       'away_last5_xGF_per_min_pp', 'away_last40_xGF_per_min_pp',
       'home_last40_GA_per_min_pk', 'home_Goalie_HDCSV%',
       'away_last5_GA_per_min_pk', 'away_last40_GF_per_min_pp',
       'away_Rating.A.Pre', 'home_last_5_xGF%_5v5', 'away_last_5_SH%',
       'home_Rating.A.Pre', 'home_last5_xGF_per_min_pp',
       'away_last_40_xGF%_5v5', 'home_last5_GA_per_min_pk',
     'away_last5_GF_per_min_pp',
       'away_last_40_GF%_5v5', 'away_last_40_SH%', 'away_last_5_FF%_5v5',
       'home_Goalie_FenwickSV%', 'away_Goalie_HDCSV%',
       'away_last40_xGA_per_min_pk', 'home_last_5_SH%',
       'away_last5_xGA_per_min_pk', 'home_last_40_FF%_5v5',
       'away_Goalie_FenwickSV%', 'away_last_40_FF%_5v5',
       'home_last40_GF_per_min_pp', 'away_last40_GA_per_min_pk']

In [14]:
X_train[numeric_features].head()

,home_last40_xGF_per_min_pp,away_last_5_xGF%_5v5,home_last5_pp_TOI_per_game,home_last_40_GF%_5v5,home_last40_xGA_per_min_pk,home_last5_xGA_per_min_pk,home_last_40_SH%,home_last5_pk_TOI_per_game,away_last40_pp_TOI_per_game,home_Goalie_GSAx/60,away_last40_pk_TOI_per_game,away_Goalie_GSAx/60,away_last_5_GF%_5v5,home_last40_pk_TOI_per_game,home_last_40_xGF%_5v5,away_last5_pp_TOI_per_game,away_last5_pk_TOI_per_game,home_last5_GF_per_min_pp,home_last_5_GF%_5v5,home_last_5_FF%_5v5,away_last5_xGF_per_min_pp,away_last40_xGF_per_min_pp,home_last40_GA_per_min_pk,home_Goalie_HDCSV%,away_last5_GA_per_min_pk,away_last40_GF_per_min_pp,away_Rating.A.Pre,home_last_5_xGF%_5v5,away_last_5_SH%,home_Rating.A.Pre,home_last5_xGF_per_min_pp,away_last_40_xGF%_5v5,home_last5_GA_per_min_pk,home_last40_pp_TOI_per_game,away_last5_GF_per_min_pp,away_last_40_GF%_5v5,away_last_40_SH%,away_last_5_FF%_5v5,home_Goalie_FenwickSV%,away_Goalie_HDCSV%,away_last40_xGA_per_min_pk,home_last_5_SH%,away_last5_xGA_per_min_pk,home_last_40_FF%_5v5,away_Goalie_FenwickSV%,away_last_40_FF%_5v5,home_last40_GF_per_min_pp,away_last40_GA_per_min_pk
0,0.112699,48.770492,4.190000,50.127801,0.104858,0.098556,9.025236,3.693333,4.646667,-0.202922,4.540000,0.082345,45.937500,4.923333,48.992719,5.893333,3.070000,0.095465,57.080799,52.399869,0.069910,0.122400,0.137102,0.858462,0.195440,0.139885,1500.66,51.663405,6.967375,1495.03,0.079714,49.339386,0.054152,5.328333,0.101810,51.399425,8.124451,52.562502,0.937294,0.873171,0.133976,9.426112,0.074267,48.803377,0.942516,49.991679,0.117297,0.121145
1,0.124909,51.204482,3.336667,56.868932,0.129028,0.153383,9.060588,3.546667,4.315417,0.169541,4.928750,-0.239655,49.927641,4.774167,51.954595,6.000000,4.966667,0.299700,59.064609,42.564205,0.096000,0.102018,0.104730,0.877358,0.040268,0.115864,1535.17,46.860987,11.358025,1577.10,0.143856,52.486645,0.225564,4.705417,0.100000,58.184556,8.420932,46.882217,0.941904,0.864516,0.097844,12.093988,0.109128,50.828439,0.941294,50.633643,0.138139,0.086229
2,0.132248,40.305523,6.283333,56.575634,0.116445,0.131278,9.025460,4.540000,4.921667,0.302087,5.185417,-0.097423,45.427286,4.233750,49.851785,4.816667,5.853333,0.190981,58.385392,60.511924,0.153218,0.120843,0.112194,0.897778,0.068337,0.116830,1496.85,60.180542,9.286882,1522.11,0.113316,49.136336,0.132159,4.682500,0.166090,50.499508,7.879167,43.520998,0.942492,0.878613,0.107127,8.478124,0.112415,50.407241,0.938246,50.595552,0.149493,0.106067
3,0.105738,49.941995,4.620000,53.260259,0.120913,0.137299,7.970138,4.763333,5.571250,-0.164139,5.305000,-0.080476,56.272661,4.379167,52.809227,5.173333,5.963333,0.043290,57.771883,54.316401,0.137242,0.143998,0.125595,0.869266,0.100615,0.103208,1496.86,52.571429,6.524847,1525.37,0.118615,50.855171,0.125962,4.778333,0.115979,45.246898,5.932286,51.909534,0.934447,0.848000,0.093779,9.804628,0.086864,52.890654,0.938305,51.197815,0.099407,0.131951
4,0.129293,43.637300,2.690000,48.882718,0.084868,0.067197,7.303942,5.446667,4.720833,-0.310233,4.475833,-0.346771,52.130045,5.193333,54.871795,6.066667,3.630000,0.297398,48.959081,52.400715,0.142088,0.087855,0.101091,0.830721,0.000000,0.121801,1545.81,50.929752,7.311321,1521.29,0.098885,50.381002,0.036720,4.482083,0.065934,52.122642,7.885816,47.102597,0.933383,0.839117,0.102718,5.518246,0.107438,55.762037,0.939698,51.309591,0.189644,0.128468


In [15]:
scoring = ['neg_log_loss', 'accuracy']

In [24]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

### Logistic Regression

In [30]:
log_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('logisticregression', LogisticRegression(max_iter=10000))])

log_params = {'logisticregression__solver' : ['liblinear', 'lbfgs', 'newton-cg'],
                'logisticregression__penalty': ['l1', 'l2'],
                'logisticregression__C': [.00001, .0001, .001, .01, .05, 0.1],
                'logisticregression__class_weight': [None] }

log_cv = GridSearchCV(log_pipeline, param_grid=log_params, cv=5, scoring=scoring, refit = 'neg_log_loss',  verbose=1)

In [31]:
log_cv.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last40_xGF_per_min_pp',
                                                                          'away_last_5_xGF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last40_xGA_per_min_pk',
                                                                          'home_last5_xGA_per_min_pk',
                                                                          'home_last_40_SH%',
                                 

In [32]:
log_cv.best_score_

-0.6754370089204439

In [33]:
log_results = pd.DataFrame(log_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
log_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logisticregression__C,param_logisticregression__class_weight,param_logisticregression__penalty,param_logisticregression__solver,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
16,0.012287,0.000163,0.007713,0.000084,0.001,None,l2,lbfgs,"{'logisticregression__C': 0.001, 'logisticregr...",-0.678374,-0.671673,-0.677392,-0.675825,-0.673921,-0.675437,0.002411,1,0.566248,0.592748,0.594972,0.571229,0.578212,0.580682,0.011433,2
17,0.019093,0.000621,0.007772,0.000036,0.001,None,l2,newton-cg,"{'logisticregression__C': 0.001, 'logisticregr...",-0.678374,-0.671674,-0.677392,-0.675825,-0.673923,-0.675437,0.002410,2,0.566248,0.592748,0.594972,0.571229,0.578212,0.580682,0.011433,2
15,0.014589,0.000732,0.008238,0.000482,0.001,None,l2,liblinear,"{'logisticregression__C': 0.001, 'logisticregr...",-0.678653,-0.672743,-0.678873,-0.676277,-0.675154,-0.676340,0.002285,3,0.567643,0.619247,0.594972,0.585196,0.569832,0.587378,0.018843,1
21,0.017617,0.000735,0.007767,0.000023,0.01,None,l2,liblinear,"{'logisticregression__C': 0.01, 'logisticregre...",-0.677641,-0.668417,-0.680201,-0.678592,-0.676997,-0.676369,0.004120,4,0.584379,0.598326,0.587989,0.565642,0.565642,0.580396,0.012887,4
23,0.025454,0.001999,0.008297,0.000467,0.01,None,l2,newton-cg,"{'logisticregression__C': 0.01, 'logisticregre...",-0.677863,-0.668443,-0.679819,-0.679093,-0.676975,-0.676439,0.004116,5,0.585774,0.594142,0.587989,0.557263,0.569832,0.579000,0.013510,6
22,0.016674,0.000611,0.007799,0.000178,0.01,None,l2,lbfgs,"{'logisticregression__C': 0.01, 'logisticregre...",-0.677864,-0.668444,-0.679819,-0.679094,-0.676973,-0.676439,0.004116,6,0.585774,0.594142,0.587989,0.557263,0.569832,0.579000,0.013510,6
24,0.017753,0.000853,0.007911,0.000061,0.05,None,l1,liblinear,"{'logisticregression__C': 0.05, 'logisticregre...",-0.678655,-0.671507,-0.677800,-0.679098,-0.675913,-0.676595,0.002768,7,0.567643,0.588563,0.596369,0.561453,0.575419,0.577889,0.012936,8
30,0.025024,0.004794,0.009411,0.000891,0.1,None,l1,liblinear,"{'logisticregression__C': 0.1, 'logisticregres...",-0.677719,-0.669147,-0.678903,-0.679826,-0.677732,-0.676665,0.003841,8,0.570432,0.596932,0.585196,0.561453,0.554469,0.573696,0.015507,13
27,0.024262,0.001168,0.008714,0.000813,0.05,None,l2,liblinear,"{'logisticregression__C': 0.05, 'logisticregre...",-0.677728,-0.669121,-0.681685,-0.681348,-0.679935,-0.677963,0.004635,9,0.581590,0.599721,0.579609,0.561453,0.555866,0.575648,0.015642,10
28,0.022607,0.000736,0.008461,0.000296,0.05,None,l2,lbfgs,"{'logisticregression__C': 0.05, 'logisticregre...",-0.677767,-0.669153,-0.681584,-0.681528,-0.679915,-0.677990,0.004632,10,0.577406,0.598326,0.581006,0.562849,0.555866,0.575091,0.014830,11


#### Feature Importance Evaluation

In [88]:
log_coef = pd.DataFrame(list(zip(X_train.columns, log_cv.best_estimator_[1].coef_[0])), columns = ['Feature', 'Coef'] )
log_coef['Coef_abs'] = abs(log_coef['Coef'])
log_coef.sort_values('Coef_abs', ascending = False)

,Feature,Coef,Coef_abs
19,home_Rating.A.Pre,-0.057949,0.057949
22,home_last_5_GF%_5v5,0.056853,0.056853
37,away_Goalie_HDCSV%,-0.048104,0.048104
35,away_Goalie_FenwickSV%,0.047458,0.047458
9,home_last_40_GF%_5v5,0.045442,0.045442
24,home_last_40_SH%,-0.039534,0.039534
27,home_last40_xGF_per_min_pp,-0.036940,0.036940
15,away_last40_xGA_per_min_pk,-0.036372,0.036372
30,home_last40_GF_per_min_pp,0.036092,0.036092
38,away_last_5_GF%_5v5,0.032656,0.032656


### Ada Boost

In [34]:
ada_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('ada', AdaBoostClassifier())])

ada_params = {'ada__n_estimators': [25, 50],
         'ada__learning_rate': [.1, 1, 10, 20],
         'ada__base_estimator': [svm.SVC(probability=True , kernel='linear'), LogisticRegression()],}

ada_cv = GridSearchCV(ada_pipeline, param_grid=ada_params, cv=5, scoring=scoring, refit='neg_log_loss', verbose=1)

In [35]:
ada_cv.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last40_xGF_per_min_pp',
                                                                          'away_last_5_xGF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last40_xGA_per_min_pk',
                                                                          'home_last5_xGA_per_min_pk',
                                                                          'home_last_40_SH%',
                                 

In [41]:
ada_cv.best_score_

-0.6799359662807147

In [36]:
ada_results = pd.DataFrame(ada_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
ada_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ada__base_estimator,param_ada__learning_rate,param_ada__n_estimators,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
4,42.529209,0.277327,2.543278,0.006013,"SVC(kernel='linear', probability=True)",10,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.683305,-0.674921,-0.681226,-0.681435,-0.678793,-0.679936,0.002889,1,0.553696,0.596932,0.583799,0.561453,0.569832,0.573142,0.015526,4
0,43.341996,0.235797,2.495557,0.040371,"SVC(kernel='linear', probability=True)",0.1,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.682869,-0.679884,-0.681938,-0.681413,-0.681377,-0.681496,0.000969,2,0.560669,0.569038,0.564246,0.551676,0.553073,0.559740,0.006589,8
5,83.132445,1.405995,5.013317,0.237507,"SVC(kernel='linear', probability=True)",10,50,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.684522,-0.676831,-0.682880,-0.684050,-0.680332,-0.681723,0.002844,3,0.557880,0.591353,0.585196,0.554469,0.567039,0.571187,0.014674,6
6,41.064554,1.219384,2.431893,0.203562,"SVC(kernel='linear', probability=True)",20,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.684800,-0.678553,-0.682937,-0.684372,-0.680497,-0.682232,0.002375,4,0.545328,0.594142,0.571229,0.546089,0.569832,0.565324,0.018196,7
8,0.122800,0.002372,0.015609,0.000248,LogisticRegression(),0.1,25,"{'ada__base_estimator': LogisticRegression(), ...",-0.684109,-0.681523,-0.684027,-0.682690,-0.682635,-0.682997,0.000969,5,0.564854,0.594142,0.597765,0.569832,0.581006,0.581520,0.012945,2
1,81.938708,0.095818,4.740888,0.009278,"SVC(kernel='linear', probability=True)",0.1,50,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.685502,-0.683732,-0.684680,-0.685002,-0.684744,-0.684732,0.000578,6,0.543933,0.542538,0.537709,0.540503,0.546089,0.542155,0.002873,12
7,73.917730,4.193198,3.965984,0.348138,"SVC(kernel='linear', probability=True)",20,50,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.686802,-0.683494,-0.685372,-0.686456,-0.683155,-0.685056,0.001494,7,0.543933,0.545328,0.540503,0.543296,0.561453,0.546902,0.007443,9
9,0.229651,0.005856,0.022538,0.000169,LogisticRegression(),0.1,50,"{'ada__base_estimator': LogisticRegression(), ...",-0.686993,-0.685253,-0.687152,-0.686154,-0.686353,-0.686381,0.000677,8,0.569038,0.602510,0.597765,0.572626,0.571229,0.582634,0.014416,1
2,35.319119,0.179226,2.073445,0.010988,"SVC(kernel='linear', probability=True)",1,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.688066,-0.688023,-0.687880,-0.688809,-0.687754,-0.688106,0.000368,9,0.543933,0.543933,0.543296,0.543296,0.544693,0.543830,0.000517,10
3,68.084216,0.296903,4.008086,0.018197,"SVC(kernel='linear', probability=True)",1,50,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.688713,-0.688734,-0.688831,-0.688501,-0.688628,-0.688681,0.000111,10,0.543933,0.543933,0.543296,0.543296,0.544693,0.543830,0.000517,10


### Gradient Boosting

In [37]:
gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('gb', GradientBoostingClassifier())])

gb_params = {'gb__n_estimators': [200, 300, 400],
         'gb__learning_rate': [.001,.01, .1],
         'gb__max_depth' : [3,5]}

gb_cv = GridSearchCV(gb_pipeline, param_grid=gb_params, cv=5, scoring=scoring, refit='neg_log_loss', verbose=1)

In [38]:
gb_cv.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last40_xGF_per_min_pp',
                                                                          'away_last_5_xGF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last40_xGA_per_min_pk',
                                                                          'home_last5_xGA_per_min_pk',
                                                                          'home_last_40_SH%',
                                 

In [39]:
gb_cv.best_score_

-0.6813351464598496

In [40]:
gb_results = pd.DataFrame(gb_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
gb_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gb__learning_rate,param_gb__max_depth,param_gb__n_estimators,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
6,5.060808,0.005743,0.013046,0.000360,0.01,3,200,"{'gb__learning_rate': 0.01, 'gb__max_depth': 3...",-0.682658,-0.679488,-0.684309,-0.680543,-0.679678,-0.681335,0.001864,1,0.559275,0.570432,0.590782,0.568436,0.582402,0.574265,0.011067,1
7,7.613778,0.132845,0.014969,0.000935,0.01,3,300,"{'gb__learning_rate': 0.01, 'gb__max_depth': 3...",-0.682625,-0.679836,-0.685340,-0.681250,-0.680842,-0.681979,0.001904,2,0.550907,0.557880,0.586592,0.565642,0.579609,0.568126,0.013270,3
8,10.091404,0.022878,0.015978,0.000220,0.01,3,400,"{'gb__learning_rate': 0.01, 'gb__max_depth': 3...",-0.682338,-0.681422,-0.686645,-0.682562,-0.682085,-0.683010,0.001857,3,0.549512,0.559275,0.585196,0.572626,0.574022,0.568126,0.012419,3
9,8.066107,0.019059,0.016398,0.000273,0.01,5,200,"{'gb__learning_rate': 0.01, 'gb__max_depth': 5...",-0.683842,-0.681349,-0.688062,-0.682242,-0.684823,-0.684064,0.002336,4,0.559275,0.570432,0.571229,0.581006,0.562849,0.568958,0.007531,2
2,10.050758,0.085446,0.018590,0.000354,0.001,3,400,"{'gb__learning_rate': 0.001, 'gb__max_depth': ...",-0.685737,-0.683565,-0.686076,-0.685384,-0.684671,-0.685087,0.000892,5,0.538354,0.548117,0.539106,0.551676,0.541899,0.543831,0.005215,13
5,15.995703,0.058481,0.026158,0.001207,0.001,5,400,"{'gb__learning_rate': 0.001, 'gb__max_depth': ...",-0.685431,-0.682782,-0.689631,-0.685636,-0.684904,-0.685677,0.002221,6,0.536960,0.560669,0.526536,0.544693,0.547486,0.543269,0.011335,14
4,11.986876,0.043948,0.021257,0.000472,0.001,5,300,"{'gb__learning_rate': 0.001, 'gb__max_depth': ...",-0.686009,-0.683521,-0.688014,-0.686682,-0.685319,-0.685909,0.001489,7,0.531381,0.560669,0.533520,0.537709,0.547486,0.542153,0.010785,16
1,7.521795,0.034738,0.015967,0.000136,0.001,3,300,"{'gb__learning_rate': 0.001, 'gb__max_depth': ...",-0.686319,-0.684856,-0.686472,-0.686419,-0.685634,-0.685940,0.000621,8,0.542538,0.545328,0.539106,0.543296,0.543296,0.542713,0.002028,15
10,12.228661,0.028340,0.019323,0.000183,0.01,5,300,"{'gb__learning_rate': 0.01, 'gb__max_depth': 5...",-0.683065,-0.684688,-0.690831,-0.686171,-0.688825,-0.686716,0.002797,9,0.566248,0.560669,0.567039,0.575419,0.564246,0.566724,0.004873,6
3,7.987818,0.031277,0.016568,0.000518,0.001,5,200,"{'gb__learning_rate': 0.001, 'gb__max_depth': ...",-0.686886,-0.685194,-0.688012,-0.688228,-0.685266,-0.686717,0.001297,10,0.535565,0.546722,0.536313,0.540503,0.550279,0.541876,0.005775,17


It does not seem that gradient boosting is producing good results for this dataset

### Neural Network

In [276]:
def build_model(optimizer='adam', activation='relu', neurons = 1, dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_dim=44, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation=activation))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [280]:
# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]

param_grid = {'nn__epochs': [8,10, 12, 15, 18],
             'nn__optimizer' : ['RMSprop', 'Adam'], 
             'nn__activation' : ['sigmoid', 'hard_sigmoid', 'linear'],
            'nn__neurons' : [12, 18, 24, 30, 36, 40],
             'nn__weight_constraint': [1, 3, 5],
             'nn__dropout_rate' : [0.0,  0.3, 0.6, 0.9]}
keras_model = scikit_learn.KerasClassifier(build_fn=build_model, verbose=0)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

nn_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('nn', keras_model)])





nn_cv = GridSearchCV(estimator=nn_pipeline, param_grid=param_grid, cv=3, scoring=scoring, refit='neg_log_loss', verbose=1)

In [ ]:
nn_cv.fit(X_train, y_train)

In [284]:
nn_results = pd.DataFrame(nn_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
nn_results.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nn__activation,param_nn__dropout_rate,param_nn__epochs,param_nn__neurons,param_nn__optimizer,param_nn__weight_constraint,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
1648,0.683245,0.006266,0.104092,0.001678,linear,0.3,8,36,Adam,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.667990,-0.671304,-0.681146,-0.673480,0.005587,1,0.592965,0.582077,0.568677,0.581240,0.009933,572
1732,0.921380,0.005455,0.104325,0.000706,linear,0.3,15,12,Adam,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.671125,-0.671530,-0.678915,-0.673857,0.003580,2,0.587940,0.598827,0.570352,0.585706,0.011732,48
1828,0.678168,0.001133,0.103484,0.001109,linear,0.6,8,36,Adam,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669233,-0.672437,-0.680127,-0.673932,0.004571,3,0.597990,0.593802,0.556114,0.582635,0.018831,333
1683,0.749562,0.003505,0.103456,0.001720,linear,0.3,10,36,Adam,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669233,-0.672677,-0.679914,-0.673941,0.004451,4,0.591290,0.589615,0.548576,0.576494,0.019752,1258
1147,0.874522,0.008508,0.112667,0.000833,hard_sigmoid,0.6,10,40,RMSprop,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.670200,-0.672712,-0.678995,-0.673969,0.003699,5,0.588777,0.592965,0.572027,0.584590,0.009046,112
1967,1.246649,0.069900,0.114459,0.003921,linear,0.6,18,30,Adam,5,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.670564,-0.672294,-0.679264,-0.674041,0.003760,6,0.578727,0.586265,0.560302,0.575098,0.010906,1375
1249,1.408697,0.335738,0.113080,0.001698,hard_sigmoid,0.6,18,36,RMSprop,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.671396,-0.671077,-0.679764,-0.674079,0.004022,7,0.583752,0.601340,0.558626,0.581240,0.017528,549
1757,0.939249,0.004742,0.103511,0.000758,linear,0.3,15,36,Adam,5,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669046,-0.672297,-0.680904,-0.674082,0.005003,8,0.596315,0.597152,0.555276,0.582915,0.019546,292
1916,0.996610,0.017258,0.106048,0.002964,linear,0.6,15,18,RMSprop,5,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.670960,-0.671626,-0.680003,-0.674196,0.004115,9,0.598827,0.598827,0.558626,0.585427,0.018951,66
1873,0.871623,0.010311,0.104613,0.000707,linear,0.6,12,12,RMSprop,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.671955,-0.671238,-0.679398,-0.674197,0.003689,10,0.582915,0.599665,0.557789,0.580123,0.017209,738


In [110]:
X_test.head()

,away_last40_GF_per_min_pp,away_last_40_FF%_5v5,away_last5_xGA_per_min_pk,home_Goalie_FenwickSV%,home_last_5_SH%,home_last5_GA_per_min_pk,home_last5_xGA_per_min_pk,away_last_40_GF%_5v5,away_last5_xGF_per_min_pp,home_last_40_GF%_5v5,away_last_5_FF%_5v5,home_Goalie_HDCSV%,home_last_40_FF%_5v5,home_last40_xGA_per_min_pk,B2B_Status,away_last40_xGA_per_min_pk,home_last40_GA_per_min_pk,home_last_5_xGF%_5v5,home_last5_GF_per_min_pp,home_Rating.A.Pre,away_last_5_SH%,away_last5_GA_per_min_pk,home_last_5_GF%_5v5,away_Goalie_GSAx/60,home_last_40_SH%,away_last_40_xGF%_5v5,home_last_40_xGF%_5v5,home_last40_xGF_per_min_pp,away_last40_GA_per_min_pk,away_last_5_xGF%_5v5,home_last40_GF_per_min_pp,away_last_40_SH%,away_last40_xGF_per_min_pp,home_last_5_FF%_5v5,away_last5_GF_per_min_pp,away_Goalie_FenwickSV%,home_last5_xGF_per_min_pp,away_Goalie_HDCSV%,away_last_5_GF%_5v5,away_Rating.A.Pre,home_Goalie_GSAx/60
3626,0.143045,49.490424,0.099403,0.936798,11.116202,0.125852,0.085579,49.414366,0.099288,55.248131,52.420611,0.869565,51.037671,0.088716,Neither,0.094008,0.112016,49.383378,0.135211,1531.48,12.303834,0.089552,70.852804,-0.291447,10.024139,49.580182,50.833631,0.104625,0.111851,50.666667,0.134196,8.701463,0.122230,45.981465,0.100545,0.937063,0.184563,0.834711,51.402501,1514.01,-0.097290
3627,0.090023,54.275967,0.084000,0.936070,4.301552,0.041580,0.121414,49.447384,0.127152,51.613248,52.992021,0.866051,50.926333,0.095972,Neither,0.115594,0.117612,50.132908,0.059465,1516.52,7.762699,0.112500,38.219070,-0.444146,9.385230,54.826724,52.907871,0.102876,0.112607,51.670507,0.161440,6.807514,0.093567,49.602278,0.000000,0.930668,0.109118,0.862233,45.479328,1497.57,-0.305563
3628,0.084120,49.003131,0.108808,0.941531,9.037111,0.139535,0.126279,51.252598,0.079574,60.496256,54.862177,0.876847,53.085863,0.101065,Neither,0.103226,0.106525,45.294475,0.107978,1584.31,10.027155,0.103627,60.267559,-0.589420,9.975409,47.514358,55.396941,0.095347,0.089670,53.023663,0.098296,9.212376,0.107010,45.077430,0.085106,0.923432,0.070186,0.857809,61.850921,1492.13,-0.023160
3629,0.128256,47.274508,0.148426,0.938316,8.090750,0.093604,0.177847,48.647099,0.108124,47.474864,50.476839,0.867299,48.540721,0.114551,Neither,0.110110,0.103250,40.077257,0.148148,1490.93,9.177190,0.134933,54.709532,-0.408527,8.348491,46.039131,48.384948,0.111322,0.116931,46.782178,0.154044,8.973817,0.098805,42.512283,0.132939,0.927771,0.097037,0.852194,46.931408,1510.30,-0.009204
3630,0.161435,53.469034,0.100000,0.938738,7.875987,0.074488,0.094972,58.414704,0.117617,55.777256,51.059929,0.852300,54.173584,0.115045,Neither,0.124851,0.091751,56.002144,0.220913,1549.43,4.253558,0.102564,38.859857,-0.058439,8.037510,53.596844,54.433969,0.107819,0.142055,49.845393,0.118953,8.848351,0.128311,53.025519,0.155440,0.936047,0.090133,0.891566,51.400000,1526.11,-0.054610


### Neural Network Part 2

In [287]:
X_nn_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r_5_40]
y_nn_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_nn_test = df[df['Season'] == '2020-2021'].loc[:,r_5_40]
y_nn_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

X_nn_pure_train, X_nn_val, y_nn_pure_train,  y_nn_val = train_test_split(X_nn_train, y_nn_train, random_state=2021, test_size=0.2 )

In [288]:
y_nn_pure_train.shape

(2865,)

In [289]:
y_nn_val.shape

(717,)

In [290]:
numeric_features = ['home_last40_xGF_per_min_pp', 'away_last_5_xGF%_5v5',
       'home_last_40_GF%_5v5',
       'home_last40_xGA_per_min_pk', 'home_last5_xGA_per_min_pk',
       'home_last_40_SH%', 
       'home_Goalie_GSAx/60',
        'away_Goalie_GSAx/60',
       'away_last_5_GF%_5v5', 
       'home_last_40_xGF%_5v5', 
     'home_last5_GF_per_min_pp',
       'home_last_5_GF%_5v5', 'home_last_5_FF%_5v5',
       'away_last5_xGF_per_min_pp', 'away_last40_xGF_per_min_pp',
       'home_last40_GA_per_min_pk', 'home_Goalie_HDCSV%',
       'away_last5_GA_per_min_pk', 'away_last40_GF_per_min_pp',
       'away_Rating.A.Pre', 'home_last_5_xGF%_5v5', 'away_last_5_SH%',
       'home_Rating.A.Pre', 'home_last5_xGF_per_min_pp',
       'away_last_40_xGF%_5v5', 'home_last5_GA_per_min_pk',
     'away_last5_GF_per_min_pp',
       'away_last_40_GF%_5v5', 'away_last_40_SH%', 'away_last_5_FF%_5v5',
       'home_Goalie_FenwickSV%', 'away_Goalie_HDCSV%',
       'away_last40_xGA_per_min_pk', 'home_last_5_SH%',
       'away_last5_xGA_per_min_pk', 'home_last_40_FF%_5v5',
       'away_Goalie_FenwickSV%', 'away_last_40_FF%_5v5',
       'home_last40_GF_per_min_pp', 'away_last40_GA_per_min_pk']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

tp = Pipeline(steps=[('preprocessor', preprocessor)])

In [291]:
X_nn_pure_train_scaled = tp.fit_transform(X_nn_pure_train)
X_nn_val_scaled = tp.transform(X_nn_val)
X_nn_test_scaled = tp.transform(X_nn_test)

In [300]:
model = Sequential()
model.add(Dense(36, activation='linear', input_dim=44, kernel_constraint=maxnorm(3)))
model.add(Dropout(.3))
model.add(Dense(4, activation='linear'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 40 Game Rolling

I will run some models using only the rolling 40 game team stats

In [305]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,r40]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [306]:
X_train.columns

Index(['home_last_40_FF%_5v5', 'home_last_40_GF%_5v5', 'home_last_40_xGF%_5v5',
       'home_last_40_SH%', 'home_last40_xGF_per_min_pp',
       'home_last40_GF_per_min_pp', 'home_last40_xGA_per_min_pk',
       'home_last40_GA_per_min_pk', 'away_last_40_FF%_5v5',
       'away_last_40_GF%_5v5', 'away_last_40_xGF%_5v5', 'away_last_40_SH%',
       'away_last40_xGF_per_min_pp', 'away_last40_GF_per_min_pp',
       'away_last40_xGA_per_min_pk', 'away_last40_GA_per_min_pk',
       'home_Goalie_FenwickSV%', 'home_Goalie_GSAx/60', 'home_Goalie_HDCSV%',
       'away_Goalie_FenwickSV%', 'away_Goalie_GSAx/60', 'away_Goalie_HDCSV%',
       'home_Rating.A.Pre', 'away_Rating.A.Pre', 'B2B_Status'],
      dtype='object')

In [307]:
numeric_features =['home_last_40_FF%_5v5', 'home_last_40_GF%_5v5', 'home_last_40_xGF%_5v5',
       'home_last_40_SH%', 'home_last40_xGF_per_min_pp',
       'home_last40_GF_per_min_pp', 'home_last40_xGA_per_min_pk',
       'home_last40_GA_per_min_pk', 'away_last_40_FF%_5v5',
       'away_last_40_GF%_5v5', 'away_last_40_xGF%_5v5', 'away_last_40_SH%',
       'away_last40_xGF_per_min_pp', 'away_last40_GF_per_min_pp',
       'away_last40_xGA_per_min_pk', 'away_last40_GA_per_min_pk',
       'home_Goalie_FenwickSV%', 'home_Goalie_GSAx/60', 'home_Goalie_HDCSV%',
       'away_Goalie_FenwickSV%', 'away_Goalie_GSAx/60', 'away_Goalie_HDCSV%',
       'home_Rating.A.Pre', 'away_Rating.A.Pre']

### Logistic Regression

In [63]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

log_40_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('logisticregression', LogisticRegression(max_iter=10000))])

In [64]:
log_params = {'logisticregression__solver' : ['liblinear', 'lbfgs', 'newton-cg'],
                'logisticregression__penalty': ['l1', 'l2'],
                'logisticregression__C': [.01, 0.1, 1, 10],
                'logisticregression__class_weight': [None] }

log_cv_40 = GridSearchCV(log_40_pipeline, param_grid=log_params, cv=5, scoring=scoring, refit = 'neg_log_loss',  verbose=1)

In [65]:
log_cv_40.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last_40_FF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last_40_xGF%_5v5',
                                                                          'home_last_40_SH%',
                                                                          'home_last40_xGF_per_min_pp',
                                                                          'home_last40_GF_per_min_pp',
                                      

In [212]:
log_40_results = pd.DataFrame(log_cv_40.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
log_40_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logisticregression__C,param_logisticregression__class_weight,param_logisticregression__penalty,param_logisticregression__solver,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
4,0.014693,0.001130,0.007716,0.000719,0.01,None,l2,lbfgs,"{'logisticregression__C': 0.01, 'logisticregre...",-0.675677,-0.672521,-0.678186,-0.674674,-0.669348,-0.674081,0.002986,1,0.570667,0.593333,0.592000,0.569333,0.581333,0.581333,0.010154,3
5,0.017283,0.000292,0.006951,0.000118,0.01,None,l2,newton-cg,"{'logisticregression__C': 0.01, 'logisticregre...",-0.675677,-0.672520,-0.678187,-0.674673,-0.669355,-0.674082,0.002984,2,0.570667,0.593333,0.592000,0.569333,0.581333,0.581333,0.010154,3
3,0.014061,0.000746,0.008523,0.001029,0.01,None,l2,liblinear,"{'logisticregression__C': 0.01, 'logisticregre...",-0.675375,-0.672548,-0.678078,-0.674946,-0.669716,-0.674133,0.002821,3,0.568000,0.596000,0.592000,0.573333,0.584000,0.582667,0.010667,2
9,0.014426,0.000387,0.006950,0.000158,0.1,None,l2,liblinear,"{'logisticregression__C': 0.1, 'logisticregres...",-0.674011,-0.672891,-0.678995,-0.674600,-0.671304,-0.674360,0.002575,4,0.570667,0.584000,0.593333,0.578667,0.576000,0.580533,0.007710,5
11,0.020913,0.000814,0.007221,0.000453,0.1,None,l2,newton-cg,"{'logisticregression__C': 0.1, 'logisticregres...",-0.674061,-0.672896,-0.679035,-0.674563,-0.671252,-0.674362,0.002600,5,0.568000,0.582667,0.593333,0.581333,0.577333,0.580533,0.008202,5
10,0.015567,0.000349,0.007100,0.000190,0.1,None,l2,lbfgs,"{'logisticregression__C': 0.1, 'logisticregres...",-0.674062,-0.672900,-0.679034,-0.674562,-0.671250,-0.674362,0.002599,6,0.568000,0.582667,0.593333,0.581333,0.577333,0.580533,0.008202,5
12,0.022668,0.001238,0.007185,0.000171,1,None,l1,liblinear,"{'logisticregression__C': 1, 'logisticregressi...",-0.673682,-0.672968,-0.679327,-0.674495,-0.671486,-0.674391,0.002659,7,0.572000,0.580000,0.590667,0.577333,0.576000,0.579200,0.006288,9
15,0.015133,0.000416,0.007112,0.000265,1,None,l2,liblinear,"{'logisticregression__C': 1, 'logisticregressi...",-0.673574,-0.673067,-0.679649,-0.674600,-0.672060,-0.674590,0.002659,8,0.573333,0.582667,0.585333,0.574667,0.572000,0.577600,0.005360,10
17,0.021284,0.000212,0.007193,0.000235,1,None,l2,newton-cg,"{'logisticregression__C': 1, 'logisticregressi...",-0.673579,-0.673068,-0.679654,-0.674596,-0.672055,-0.674590,0.002661,9,0.573333,0.582667,0.585333,0.574667,0.572000,0.577600,0.005360,10
16,0.016655,0.000304,0.006913,0.000190,1,None,l2,lbfgs,"{'logisticregression__C': 1, 'logisticregressi...",-0.673578,-0.673068,-0.679656,-0.674598,-0.672055,-0.674591,0.002662,10,0.573333,0.582667,0.585333,0.574667,0.572000,0.577600,0.005360,10


### Ada Boost

In [66]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


ada_40_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('ada', AdaBoostClassifier())])

ada_params = {'ada__n_estimators': [25],
         'ada__learning_rate': [.01, .1, 1, 10],
         'ada__base_estimator': [svm.SVC(probability=True , kernel='linear'), LogisticRegression()],}

ada_cv_40 = GridSearchCV(ada_40_pipeline, param_grid=ada_params, cv=5, scoring=scoring, refit='neg_log_loss', verbose=1)

In [67]:
ada_cv_40.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last_40_FF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last_40_xGF%_5v5',
                                                                          'home_last_40_SH%',
                                                                          'home_last40_xGF_per_min_pp',
                                                                          'home_last40_GF_per_min_pp',
                                      

In [68]:
ada_40_results = pd.DataFrame(ada_cv_40.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
ada_40_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ada__base_estimator,param_ada__learning_rate,param_ada__n_estimators,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,37.925709,0.638357,2.145070,0.033783,"SVC(kernel='linear', probability=True)",0.01,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.678507,-0.672283,-0.677155,-0.678249,-0.672888,-0.675816,0.002684,1,0.550907,0.577406,0.599162,0.561453,0.579609,0.573707,0.016532,5
4,0.112370,0.003209,0.015125,0.000329,LogisticRegression(),0.01,25,"{'ada__base_estimator': LogisticRegression(), ...",-0.680541,-0.676259,-0.679670,-0.678631,-0.675985,-0.678217,0.001817,2,0.569038,0.588563,0.569832,0.567039,0.583799,0.575654,0.008774,3
3,37.559270,0.155653,2.273125,0.004889,"SVC(kernel='linear', probability=True)",10,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.681224,-0.674573,-0.677922,-0.680711,-0.676703,-0.678227,0.002487,3,0.567643,0.595537,0.608939,0.571229,0.572626,0.583195,0.016198,1
1,38.204704,0.113358,2.208953,0.011606,"SVC(kernel='linear', probability=True)",0.1,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.682440,-0.679579,-0.682155,-0.681328,-0.679784,-0.681057,0.001183,4,0.564854,0.574616,0.564246,0.568436,0.565642,0.567559,0.003809,6
5,0.114122,0.000884,0.014839,0.000070,LogisticRegression(),0.1,25,"{'ada__base_estimator': LogisticRegression(), ...",-0.684177,-0.681519,-0.684073,-0.683125,-0.681574,-0.682894,0.001160,5,0.560669,0.588563,0.597765,0.569832,0.582402,0.579847,0.013203,2
2,31.212874,0.111837,1.813163,0.012767,"SVC(kernel='linear', probability=True)",1,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.688093,-0.687867,-0.688317,-0.688659,-0.686559,-0.687899,0.000719,6,0.543933,0.543933,0.543296,0.543296,0.544693,0.543830,0.000517,7
6,0.122421,0.057093,0.015021,0.000210,LogisticRegression(),1,25,"{'ada__base_estimator': LogisticRegression(), ...",-0.691571,-0.691189,-0.691649,-0.691591,-0.691297,-0.691459,0.000182,7,0.559275,0.594142,0.589385,0.564246,0.569832,0.575376,0.013873,4
7,0.232039,0.006414,0.015102,0.000146,LogisticRegression(),10,25,"{'ada__base_estimator': LogisticRegression(), ...",-0.691897,-0.691110,-0.702624,-0.690811,-0.691471,-0.693583,0.004535,8,0.543933,0.543933,0.543296,0.543296,0.544693,0.543830,0.000517,7


### Neural Network

In [310]:
def build_model(optimizer='adam', activation='relu', neurons = 1, dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_dim=28, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation=activation))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

param_grid = {'nn__epochs': [8,10, 15, 18],
             'nn__optimizer' : ['RMSprop', 'Adam'], 
             'nn__activation' : ['hard_sigmoid', 'linear'],
            'nn__neurons' : [12, 24, 36, 40],
             'nn__weight_constraint': [1, 3],
             'nn__dropout_rate' : [0.3, 0.6]}

keras_model = scikit_learn.KerasClassifier(build_fn=build_model, verbose=0)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

nn_40_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('nn', keras_model)])





nn_40_cv = GridSearchCV(estimator=nn_40_pipeline, param_grid=param_grid, cv=3, scoring=scoring, refit='neg_log_loss', verbose=1)

In [311]:
nn_40_cv.fit(X_train, y_train)

Fitting 3 folds for each of 256 candidates, totalling 768 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last_40_FF%_5v5',
                                                                          'home_last_40_GF%_5v5',
                                                                          'home_last_40_xGF%_5v5',
                                                                          'home_last_40_SH%',
                                                                          'home_last40_xGF_per_min_pp',
                                                                          'home_last40_GF_per_min_pp',
                                      

In [314]:
nn_40_results = pd.DataFrame(nn_40_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
nn_40_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nn__activation,param_nn__dropout_rate,param_nn__epochs,param_nn__neurons,param_nn__optimizer,param_nn__weight_constraint,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
186,1.026956,0.006196,0.313454,0.299331,linear,0.3,18,36,Adam,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.670244,-0.670506,-0.677651,-0.672800,0.003431,1,0.585427,0.594640,0.564489,0.581519,0.012615,66
201,0.714656,0.003949,0.099995,0.000245,linear,0.6,8,36,RMSprop,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669582,-0.671819,-0.677286,-0.672896,0.003236,2,0.582915,0.602178,0.559464,0.581519,0.017466,66
45,1.036484,0.007792,0.110218,0.000721,hard_sigmoid,0.3,15,40,RMSprop,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.669712,-0.670576,-0.678738,-0.673009,0.004066,3,0.574539,0.595477,0.573702,0.581240,0.010073,79
182,1.004689,0.007679,0.100871,0.001236,linear,0.3,18,24,Adam,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669191,-0.671065,-0.678912,-0.673056,0.004211,4,0.585427,0.592127,0.564489,0.580681,0.011772,91
130,0.841163,0.297668,0.101427,0.001652,linear,0.3,8,12,Adam,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.671780,-0.669664,-0.677814,-0.673086,0.003453,5,0.576214,0.583752,0.563652,0.574539,0.008291,220
228,1.156157,0.303291,0.103092,0.001277,linear,0.6,15,24,RMSprop,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669714,-0.670462,-0.679106,-0.673094,0.004262,6,0.580402,0.603015,0.559464,0.580960,0.017784,84
178,0.954173,0.001785,0.100213,0.001110,linear,0.3,18,12,Adam,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.670004,-0.671762,-0.677552,-0.673106,0.003225,7,0.582077,0.595477,0.567839,0.581798,0.011285,61
123,1.107433,0.005035,0.109486,0.000659,hard_sigmoid,0.6,18,36,Adam,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.670246,-0.670674,-0.678426,-0.673115,0.003759,8,0.572864,0.595477,0.566164,0.578169,0.012541,142
155,0.733315,0.002264,0.100377,0.000298,linear,0.3,10,36,Adam,3,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.669381,-0.670897,-0.679089,-0.673122,0.004264,9,0.592127,0.592127,0.564489,0.582915,0.013029,37
144,0.770164,0.000856,0.313379,0.301552,linear,0.3,10,12,RMSprop,1,"{'nn__activation': 'linear', 'nn__dropout_rate...",-0.670948,-0.670709,-0.677717,-0.673125,0.003249,10,0.586265,0.597152,0.554439,0.579285,0.018123,120


## All Rolling Game Features With Recursive Feature Elimination

In [69]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,all_r]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,all_r]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [383]:
X_train.shape

(3750, 104)

### Recursive Feature Elimination

In [70]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

rfecv = RFECV(estimator= LogisticRegression(max_iter =10000, penalty = 'l2', solver='liblinear', C=.1), step=1, scoring='accuracy')
rfecv_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfecv', rfecv)])

In [71]:
rfecv_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['home_last_40_FF%_5v5',
                                                   'home_last_40_GF%_5v5',
                                                   'home_last_40_xGF%_5v5',
                                                   'home_last_40_SH%',
                                                   'home_last40_xGF_per_min_pp',
                                                   'home_last40_GF_per_min_pp',
                                                   'home_last40_xGA_per_min_pk',
                                                   'home_last40_GA_per_min_pk',
                                                   'away_last_40_FF%_5v5',
                                                   

In [72]:
rfecv_pipeline[1].n_features_

9

In [73]:
rfecv_pipeline[1].ranking_

array([ 1,  1, 13,  8, 18, 14,  9,  6,  2, 20, 17,  7, 10, 12, 16, 15,  1,
        1,  5, 19, 11,  4,  1,  1,  1,  1,  1,  3])

In [74]:
rfecv_results = pd.DataFrame(list(zip(X_train.columns, rfecv_pipeline[1].ranking_)), columns = ['Feature', 'Ranking']).sort_values('Ranking')
rfecv_results.head(rfecv_pipeline[1].n_features_)

,Feature,Ranking
0,home_last3_xGF_per_min_pp,1
1,home_Goalie_FenwickSV%,1
25,home_last_20_GF%_5v5,1
24,away_last_30_GF%_5v5,1
23,away_Rating.A.Pre,1
22,home_last_10_FF%_5v5,1
17,away_last10_xGF_per_min_pp,1
16,home_last_40_xGF%_5v5,1
26,away_last40_GF_per_min_pp,1


In [75]:
rfecv_columns = list(rfecv_results.iloc[:rfecv_pipeline[1].n_features_,0])
rfecv_columns 

['home_last3_xGF_per_min_pp',
 'home_Goalie_FenwickSV%',
 'home_last_20_GF%_5v5',
 'away_last_30_GF%_5v5',
 'away_Rating.A.Pre',
 'home_last_10_FF%_5v5',
 'away_last10_xGF_per_min_pp',
 'home_last_40_xGF%_5v5',
 'away_last40_GF_per_min_pp']

### Logistic Regression

In [76]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,rfecv_columns]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,rfecv_columns]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [77]:
log_rfecv_pipeline = Pipeline(steps=[('ss', StandardScaler()),
                      ('logisticregression', LogisticRegression(max_iter=10000))])

log_params = {'logisticregression__solver' : ['liblinear', 'lbfgs', 'newton-cg'],
                'logisticregression__penalty': ['l1', 'l2'],
                'logisticregression__C': [.01, 0.1, 10, 20, 100],
                'logisticregression__class_weight': [None]}

log_cv_all = GridSearchCV(log_rfecv_pipeline, param_grid=log_params, cv=5, scoring=scoring, refit = 'neg_log_loss',  verbose=1)

In [78]:
log_cv_all.fit(X_train[rfecv_columns], y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'logisticregression__C': [0.01, 0.1, 10, 20, 100],
                         'logisticregression__class_weight': [None],
                         'logisticregression__penalty': ['l1', 'l2'],
                         'logisticregression__solver': ['liblinear', 'lbfgs',
                                                        'newton-cg']},
             refit='neg_log_loss', scoring=['neg_log_loss', 'accuracy'],
             verbose=1)

In [79]:
log_all_results = pd.DataFrame(log_cv_all.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
log_all_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logisticregression__C,param_logisticregression__class_weight,param_logisticregression__penalty,param_logisticregression__solver,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
6,0.006220,0.000130,0.003461,0.000075,0.1,None,l1,liblinear,"{'logisticregression__C': 0.1, 'logisticregres...",-0.683493,-0.676186,-0.674550,-0.672781,-0.673968,-0.676196,0.003810,1,0.550907,0.581590,0.594972,0.574022,0.576816,0.575661,0.014317,17
4,0.007108,0.000076,0.003425,0.000014,0.01,None,l2,lbfgs,"{'logisticregression__C': 0.01, 'logisticregre...",-0.682996,-0.675241,-0.675597,-0.673949,-0.673375,-0.676232,0.003479,2,0.546722,0.585774,0.596369,0.565642,0.581006,0.575103,0.017297,18
5,0.013248,0.001218,0.004097,0.000478,0.01,None,l2,newton-cg,"{'logisticregression__C': 0.01, 'logisticregre...",-0.682997,-0.675241,-0.675597,-0.673949,-0.673375,-0.676232,0.003479,3,0.546722,0.585774,0.596369,0.565642,0.581006,0.575103,0.017297,18
3,0.005481,0.000175,0.003253,0.000013,0.01,None,l2,liblinear,"{'logisticregression__C': 0.01, 'logisticregre...",-0.682877,-0.675304,-0.675674,-0.674005,-0.673641,-0.676300,0.003376,4,0.555091,0.584379,0.587989,0.575419,0.579609,0.576497,0.011518,16
10,0.007337,0.000301,0.003419,0.000033,0.1,None,l2,lbfgs,"{'logisticregression__C': 0.1, 'logisticregres...",-0.684044,-0.676038,-0.675903,-0.673108,-0.673425,-0.676503,0.003961,5,0.549512,0.585774,0.592179,0.578212,0.581006,0.577337,0.014696,1
11,0.011399,0.000528,0.003445,0.000016,0.1,None,l2,newton-cg,"{'logisticregression__C': 0.1, 'logisticregres...",-0.684044,-0.676037,-0.675902,-0.673108,-0.673426,-0.676503,0.003961,6,0.549512,0.585774,0.592179,0.578212,0.581006,0.577337,0.014696,1
9,0.005766,0.000048,0.003394,0.000016,0.1,None,l2,liblinear,"{'logisticregression__C': 0.1, 'logisticregres...",-0.684022,-0.676034,-0.675904,-0.673109,-0.673453,-0.676504,0.003948,7,0.546722,0.585774,0.589385,0.579609,0.581006,0.576499,0.015289,5
12,0.006644,0.000130,0.003406,0.000029,10,None,l1,liblinear,"{'logisticregression__C': 10, 'logisticregress...",-0.684239,-0.676227,-0.675958,-0.672984,-0.673468,-0.676575,0.004045,8,0.549512,0.585774,0.589385,0.579609,0.581006,0.577057,0.014204,3
18,0.006527,0.000311,0.003445,0.000019,20,None,l1,liblinear,"{'logisticregression__C': 20, 'logisticregress...",-0.684245,-0.676233,-0.675967,-0.672985,-0.673465,-0.676579,0.004046,9,0.548117,0.585774,0.590782,0.578212,0.581006,0.576778,0.014956,4
16,0.007279,0.000221,0.003613,0.000365,10,None,l2,lbfgs,"{'logisticregression__C': 10, 'logisticregress...",-0.684249,-0.676236,-0.675975,-0.672987,-0.673460,-0.676582,0.004048,10,0.548117,0.584379,0.590782,0.578212,0.581006,0.576499,0.014798,5


### Ada Boost

In [80]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,rfecv_columns]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,rfecv_columns]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [81]:
ada_rfecv_pipeline = Pipeline(steps=[('ss', StandardScaler()),
                      ('ada', AdaBoostClassifier())])

ada_params = {'ada__n_estimators': [25],
         'ada__learning_rate': [ .1, 10],
         'ada__base_estimator': [svm.SVC(probability=True , kernel='linear'), LogisticRegression(max_iter =10000, C=.01, penalty = 'l1', solver = 'liblinear')],}

ada_cv_all = GridSearchCV(ada_rfecv_pipeline, param_grid=ada_params, cv=5, scoring=scoring, refit='neg_log_loss', verbose=1)

In [82]:
ada_cv_all.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('ada', AdaBoostClassifier())]),
             param_grid={'ada__base_estimator': [SVC(kernel='linear',
                                                     probability=True),
                                                 LogisticRegression(C=0.01,
                                                                    max_iter=10000,
                                                                    penalty='l1',
                                                                    solver='liblinear')],
                         'ada__learning_rate': [0.1, 10],
                         'ada__n_estimators': [25]},
             refit='neg_log_loss', scoring=['neg_log_loss', 'accuracy'],
             verbose=1)

In [83]:
ada_all_results = pd.DataFrame(ada_cv_all.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
ada_all_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ada__base_estimator,param_ada__learning_rate,param_ada__n_estimators,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
1,32.347400,0.550717,2.137133,0.103863,"SVC(kernel='linear', probability=True)",10,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.683982,-0.678475,-0.679754,-0.682702,-0.681416,-0.681266,1.976734e-03,1,0.556485,0.591353,0.574022,0.561453,0.555866,0.567836,0.013448,1
0,32.946891,0.287765,2.113290,0.028486,"SVC(kernel='linear', probability=True)",0.1,25,"{'ada__base_estimator': SVC(kernel='linear', p...",-0.684418,-0.682521,-0.682660,-0.681958,-0.682048,-0.682721,8.898324e-04,2,0.553696,0.564854,0.571229,0.553073,0.560056,0.560581,0.006866,2
2,0.055528,0.000309,0.010574,0.000029,"LogisticRegression(C=0.01, max_iter=10000, pen...",0.1,25,{'ada__base_estimator': LogisticRegression(C=0...,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,8.599751e-17,3,0.456067,0.456067,0.456704,0.456704,0.455307,0.456170,0.000517,3
3,0.055622,0.000853,0.010454,0.000019,"LogisticRegression(C=0.01, max_iter=10000, pen...",10,25,{'ada__base_estimator': LogisticRegression(C=0...,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,8.599751e-17,3,0.456067,0.456067,0.456704,0.456704,0.455307,0.456170,0.000517,3


### Neural Network

In [326]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,all_r]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].loc[:,all_r]
y_test = df[df['Season'] == '2020-2021']['Home_Team_Won']

In [335]:
numeric_features = list(X_train.columns)
numeric_features.remove('B2B_Status')

In [331]:
X_train.shape

(3582, 105)

In [339]:
def build_model(optimizer='adam', activation='relu', neurons = 1, dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_dim=108, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation=activation))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


param_grid = {'nn__epochs': [8,10, 15, 18],
             'nn__optimizer' : ['Adam'], 
             'nn__activation' : ['hard_sigmoid', 'linear'],
            'nn__neurons' : [12, 24, 36, 40],
             'nn__weight_constraint': [1, 3],
             'nn__dropout_rate' : [0.3, 0.6]}
keras_model = scikit_learn.KerasClassifier(build_fn=build_model, verbose=0)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['B2B_Status']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

nn_all_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('nn', keras_model)])





nn_all_cv = GridSearchCV(estimator=nn_all_pipeline, param_grid=param_grid, cv=3, scoring=scoring, refit='neg_log_loss', verbose=1)

In [340]:
nn_all_cv.fit(X_train, y_train)

Fitting 3 folds for each of 128 candidates, totalling 384 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['home_last3_xGF_per_min_pp',
                                                                          'home_Goalie_FenwickSV%',
                                                                          'home_last5_xGA_per_min_pk',
                                                                          'home_last30_xGA_per_min_pk',
                                                                          'away_last5_xGF_per_min_pp',
                                                                          'home_last20_GA_per_min_pk',
                  

In [342]:
nn_all_results = pd.DataFrame(nn_all_cv.cv_results_).sort_values('mean_test_neg_log_loss', ascending=False)
nn_all_results.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nn__activation,param_nn__dropout_rate,param_nn__epochs,param_nn__neurons,param_nn__optimizer,param_nn__weight_constraint,params,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
43,0.794308,0.002446,0.112324,0.000971,hard_sigmoid,0.6,10,24,Adam,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.674352,-0.673008,-0.681466,-0.676276,0.003711,1,0.577889,0.601340,0.551926,0.577052,0.020182,27
4,0.729534,0.005631,0.111911,0.000338,hard_sigmoid,0.3,8,36,Adam,1,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.674617,-0.672954,-0.683073,-0.676882,0.004430,2,0.583752,0.603853,0.544389,0.577331,0.024697,24
56,1.101628,0.005594,0.112252,0.000862,hard_sigmoid,0.6,18,12,Adam,1,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.675368,-0.673330,-0.682021,-0.676906,0.003711,3,0.576214,0.601340,0.548576,0.575377,0.021549,44
3,0.905045,0.253149,0.113350,0.000761,hard_sigmoid,0.3,8,24,Adam,3,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.674908,-0.673728,-0.682318,-0.676985,0.003802,4,0.582077,0.596315,0.541876,0.573423,0.023052,56
50,0.991799,0.002320,0.113802,0.003088,hard_sigmoid,0.6,15,24,Adam,1,"{'nn__activation': 'hard_sigmoid', 'nn__dropou...",-0.675537,-0.672514,-0.683368,-0.677140,0.004574,5,0.577889,0.594640,0.552764,0.575098,0.017209,47


## Apply Best Models To Test Data

I will evaluate the best model iterations on the held out 2021 season data

In [343]:
results_dict = {'cv accuracy': {}, 'cv log loss': {}, 'test accuracy': {}, 'test log_loss':{}}
accuracy_list = []
log_loss_list = []

In [344]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r_5_40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r_5_40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



test_preds_5_40 = log_cv.predict(X_test)

test_probs_5_40 = log_cv.predict_proba(X_test)


accuracy_list.append(accuracy_score(y_test, test_preds_5_40))
log_loss_list.append(log_loss(y_test, test_probs_5_40))


ValueError: X has 41 features, but ColumnTransformer is expecting 49 features as input.

In [ ]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



test_preds_40 = log_cv_40.predict(X_test)

test_probs_40 = log_cv_40.predict_proba(X_test)

accuracy_list.append(accuracy_score(y_test, test_preds_40))
log_loss_list.append(log_loss(y_test, test_probs_40))

In [ ]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,rfecv_columns]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,rfecv_columns]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']

test_preds_rfecv = log_cv_all.predict(X_test)

test_probs_rfecv = log_cv_all.predict_proba(X_test)


accuracy_list.append(accuracy_score(y_test, test_preds_rfecv))
log_loss_list.append(log_loss(y_test, test_probs_rfecv))



In [345]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r_5_40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r_5_40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



accuracy_list.append(accuracy_score(y_test, ada_cv.predict(X_test)))
log_loss_list.append(log_loss(y_test,ada_cv.predict_proba(X_test)))

ValueError: X has 41 features, but ColumnTransformer is expecting 49 features as input.

In [365]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



accuracy_list.append(accuracy_score(y_test, ada_cv_40.predict(X_test)))
log_loss_list.append(log_loss(y_test, ada_cv_40.predict_proba(X_test)))

In [ ]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r_5_40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r_5_40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



accuracy_list.append(accuracy_score(y_test, nn_cv.predict(X_test)))
log_loss_list.append(log_loss(y_test, nn_cv.predict_proba(X_test)))

In [ ]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,r40]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,r40]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



accuracy_list.append(accuracy_score(y_test, nn_40_cv.predict(X_test)))
log_loss_list.append(log_loss(y_test, nn_40_cv.predict_proba(X_test)))

In [ ]:
X_train = df[df['Season'] != '2020-2021'].dropna().loc[:,all_r]
y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
X_test = df[df['Season'] == '2020-2021'].dropna().loc[:,all_r]
y_test = df[df['Season'] == '2020-2021'].dropna()['Home_Team_Won']



accuracy_list.append(accuracy_score(y_test, nn_40_cv.predict(X_test)))
log_loss_list.append(log_loss(y_test, nn_40_cv.predict_proba(X_test)))

In [403]:
results_dict['test accuracy'] = accuracy_list
results_dict['test log_loss'] = log_loss_list
models = ['5 and 40 Logistic Regression', 
          '40 Logistic Regression', 
          'rfecv Logistic Regression', 
          '5 and 40 AdaBoost', 
          '40 AdaBoost', 
          '5 and 40 Neural Network', 
          '40 Neural Network', 
          'All Neural Network']
results_dict['cv accuracy'] = [log_results['mean_test_accuracy'][0], 
                               log_40_results['mean_test_accuracy'][0], 
                               log_all_results['mean_test_accuracy'][0], 
                               ada_results['mean_test_accuracy'][0], 
                               ada_40_results['mean_test_accuracy'][0], 
                               nn_results['mean_test_accuracy'][0],
                              nn_40_results['mean_test_accuracy'][0],
                              nn_all_results['mean_test_accuracy'][0]]
results_dict['cv log loss'] = [log_cv.best_score_*-1, 
                               log_cv_40.best_score_*-1, 
                               log_cv_all.best_score_*-1, 
                               ada_cv.best_score_*-1, 
                               ada_cv_40.best_score_*-1, 
                               nn_cv.best_score_*-1, 
                               nn_40_cv.best_score_*-1, 
                               nn_all_cv.best_score_*-1]

In [404]:
results_df = pd.DataFrame(results_dict, index = models)

## Conclusion

Best model was logistic regression with the rolling 5 and 40 features on the test data. Interestingly, this was the 4th best model on the CV training data set though it did have the best CV accuracy.

In [405]:
results_df.sort_values('test log_loss')

,cv accuracy,cv log loss,test accuracy,test log_loss
5 and 40 log,0.581067,0.677735,0.597205,0.657201
40 log,0.580267,0.674081,0.593393,0.657240
40 ada,0.579467,0.675472,0.606099,0.660695
rfecv log,0.571733,0.675783,0.590851,0.665077
5 and 40 ada,0.562667,0.681288,0.560356,0.678121


## Next Steps
To further improve the models I would like to take the following next steps

- Train a neural network model
- Categorize B2B better
- Include team ELO feature
- Try linear weightings in rolling features
- Increase goalie games
- Add prior year goalie GAR feature
- Add Team HDSC % feature
- Add more seasons to training set
- Compare against historical implied odds from a bookmaker
- Adjust ineperienced goalie imputed stats and exclude 2021 season to avoid data leakage on test set